## Taller de cuantificación: Estimación de prevalencias por clase
Pontentes: Pablo Gonzalez (gonzalezgpablo@uniovi.es) y Olaya Pérez (perezolaya@uniovi.es)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pglez82/quantificationlib_lab/blob/master/lab/lab_solucion.ipynb)

### Parte 1: Descarga del dataset necesario

In [ ]:
import os
import zipfile

zip_url = "https://zenodo.org/records/11661820/files/T1.train_dev.zip?download=1"
zip_path = "T1.train_dev.zip"
extracted_folder = "T1.train_dev"

# Download with system wget (Jupyter syntax)
if not os.path.exists(zip_path):
    print("Downloading dataset...")
    !wget -O $zip_path "$zip_url"
else:
    print("ZIP file already exists.")

# Extract if not already extracted
if not os.path.exists(extracted_folder):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

### Parte 2: Instalación de las librerías necesarias
Necesitaremos:
- **pandas**: para cargar los datos y manejarlos de manera sencilla.
- **matplotlib**: para realizar gráficos.
- **scikit-learn**: para utilizar clasificadores como por ejemplo Logistic Regression.
- **quantificationlib**: librería con los métodos de cuantificación implementados.

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install quantificationlib
!pip install scikit-learn

### Parte 3. Leyendo el dataset.
Este dataset contiene reviews de productos de amazon. Cada review puede ser positiva o negativa (campo `label`). El texto de cada review ha sido convertido a una representación numérica para su facilidad de uso. Diponemos de 5000 ejemplos de entrenamiento (5000 opiniones).

In [ ]:
import pandas as pd

dataset = pd.read_csv('T1.train_dev/T1/public/training_data.txt')

print(dataset)

### Analizando la prevalencia de las clases original en el dataset de entrenamiento.
Como podemos ver, un alto porcentaje de las opiniones son positivas.

In [ ]:
import matplotlib.pyplot as plt

# Count examples per label
label_counts = dataset['label'].value_counts()

# Plot
plt.figure(figsize=(5, 4))
plt.bar(label_counts.index.astype(str), label_counts.values)
plt.xlabel('Etiqueta (0 negativa, 1 positiva)')
plt.ylabel('Número de ejemplos')
plt.title('Distribución')

plt.grid(axis='y')
plt.show()


### Aprendiendo a cuantificar. El método Clasificar y Contar
Vamos a empezar con la solución trivial del problema de la cuantificación: entrenar un clasificador y contar las predicciones de cada ejemplo de la bag de test que queremos cuantificar.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

X = dataset.drop(columns=["label"]).values
y = dataset["label"].values

#Dividimos nuestro dataset en train y test, separando el 30% de los ejemplos para test y asegurando que la proporción original de etiquetas se mantiene en ambos conjuntos.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Train: {len(y_train)} ejemplos (Proporción positivos: {np.mean(y_train):.2f}) | Test: {len(y_test)} ejemplos (Proporción positivos: {np.mean(y_test):.2f})")

Entrenamos un clasificador y clasificamos lo ejemplos de test. Como podemos ver, el error absoluto de este cuantificador (AE) es bajo, predice bastante bien la prevalencia de la clse positiva en la bag de test. Este problema es en realidad trivial ya que en nuestro setup la prevalencia de las clases **no ha cambiado** entre train y test: $P_{train}(y)=P_{test}(y)$

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# Predeccimos los ejemplos de test y contamos la proporción de positivos en las predicciones.

# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------

print(f"Clasificar y contar (porcentaje positivos): {cc_prevalence:.4f}")
print(f"Porcentaje positivos real: {true_prevalence:.4f}")
print(f"Error absoluto cuantificador: {np.abs(cc_prevalence-true_prevalence):.4f}")

### ¿Qué sucede si la distribución $P(y)$ cambia entre train y test $P_{train}(y) \neq P_{test}(y)$?
Para probarlo, generamos una nueva bolsa de test con 500 ejemplos, pero en este caso, la mayoría van a ser negativos (400 de 500).
Como podemos comprobar ahora, si repetimos el procedimiento de antes y utilizamos CC para cuantificar, podemos comprobar como el error absoluto entre la predicción del cuantificador y la prevalencia real es mucho mayor.

📝***Hazlo tu mismo**: Escribe el código necesario para clasificar los ejemplos de test con el clasificador entrenado y contar cuantos ejemplos pertenecen a la clase positiva. Imprime también el error absoluto de esta estimación de la prevalencia $AE=|\hat{p}-p|$.*

In [ ]:
# Calculamos los índices de los ejemplos negativos y positivos en el conjunto de test
neg_indices = np.where(y_test == 0)[0]
pos_indices = np.where(y_test == 1)[0]

# Elegimos el número de ejemplos negativos y positivos que queremos en nuestra bag
n_neg, n_pos = 400, 100

# Muestreamos aleatoriamente con reemplazmiento los índices de los ejemplos negativos y positivos
sampled_neg = np.random.choice(neg_indices, size=n_neg, replace=True)
sampled_pos = np.random.choice(pos_indices, size=n_pos, replace=True)

# Combinamos y barajamos los índices de la bag
bag_indices = np.concatenate([sampled_neg, sampled_pos])
np.random.shuffle(bag_indices)

# Creamos la bag con los ejemplos seleccionados
X_bag = X_test[bag_indices]
y_bag = y_test[bag_indices]

# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------


### Automatizando la cuantificación (quantificationlib)

Hasta ahora hemos cuantificado, con un cuantificador trivial (CC) a mano. Para cuantificar disponemos de software específico, como por ejemplo la librería de cuantificación **quantificationlib** (https://aicgijon.github.io/quantificationlib/).

Veamos como utilizar CC usando quantificationlib.

In [ ]:
from quantificationlib.baselines.cc import CC
from quantificationlib.metrics.multiclass import mean_absolute_error

# Creamos un objeto CC que corresponde al método clasificar y contar (CC) e indicamos que queremos el mismo clasificador que hemos utilizado antes (clf).
quantifier_cc = CC(estimator_test=clf)
quantifier_cc.fit(X_train,y_train)
# Predeccimos la misma bag con el cuantificador CC (deberíamos de obtener los mismos resultados que antes)
p_pred=quantifier_cc.predict(X_bag)

true_prevalence = np.array([0.8,0.2])

print(f"Clasificar y contar (porcentaje postivos): {p_pred[1]}")
print(f"Prevalencia real: {true_prevalence[1]}")
# También disponemos de funciones para calcular errores de cuantificación como el error absoluto
print(f"Error absoluto cuantificador: {mean_absolute_error(p_pred,true_prevalence)}")

### Utilizando cuantificadores más avanzados
En este caso vamos a utilizar un cuantificador especialmente diseñador para lidiar con situaciones de prior probability shift. Este método es un cuantificador básico conocido como Adjusted Count (AC).

Como podemos ver este cuantificador mejora muchísimo el resultado de CC.

📝 ***Hazlo tu mismo**: Añade el código necesario para cuantificar usando **AC** implementado en quantificationlib e imprimir la prevalencia obtenida y el error cometido. Ten en cuenta que AC necesita un estimador de train (para estimar el tpr y el fpr, y un estimador de test. Estos dos clasificadores suelen ser el mismo.* 

In [ ]:
from quantificationlib.baselines.ac import AC

# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------

### Evaluando correctamente un cuantificador

Hasta ahora hemos hecho nuestras pruebas sobre una sola bag, con una prevalencia fija. Pero, no sería más adecuado crear varias bags con prior probability shift y calcular el error que cometemos? Esto se puede realizar fácilmente con un generador de bags incluido en **quantificationlib**. Veamos como se usa.

📝 ***Hazlo tú mismo**: Utilizando `prev_true` e `indexes` devuelto, evalúa los cuantificadores CC y AC en cada una de las bags, calculando el error absoluto medio para las 50 bags. `prev_true` tendrá una forma (n_classes, n_bags) e `indexes` tendrá una forma (bag_size, n_bags).*

In [ ]:
from quantificationlib.bag_generator import PriorShift_BagGenerator

n_bags = 50
bag_size = 500

bag_generator = PriorShift_BagGenerator(n_bags=n_bags, bag_size=bag_size, min_prevalence=None)
# prev_true es la prevalencia real de cada bag y indexes son los índices de los ejemplos de test que pertenecen a cada bag.
# prev_true tiene forma (2, n_bags) y indexes tiene forma (bag_size, n_bags)
prev_true, indexes = bag_generator.generate_bags(X_test, y_test)

# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------


### Visualizandolo gráficamente
Otra manera de ver el rendimiento de los cuantificadores es visualizarlo gráficamente. Podemos pintar en un gráfico de dos dimesiones, donde el eje X representa la prevalencia real y el eje Y representa la prevalencia estimada. La linea x=y representaría el cuantificador perfecto (aquel que es capaz de predecir la prevalencia real de la bag).

In [ ]:
prev_true = prev_true[1, :]  # Prevalencia real de la clase positiva
preds_cc = preds_cc[1, :]  # Predicciones del cuantificador CC
preds_ac = preds_ac[1, :]  # Predicciones del cuantificador AC

plt.figure(figsize=(8, 6))
plt.plot([0, 1], [0, 1], 'k--', label='Cuantificador perfecto (y = x)')  # línea y=x
plt.scatter(prev_true, preds_cc, color='green', label='CC', marker='o')
plt.scatter(prev_true, preds_ac, color='red', label='AC', marker='s')

# Etiquetas y leyenda
plt.xlabel('Prevalencia real')
plt.ylabel('Prevalencia estimada')
plt.title('Comparación de predicciones de cuantificadores')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Ejercicio 1: Entrena dos cuantificadores más. El primero basado en distribution matching (DFy) y el segundo basado en el agoritmo EM (EM), compara los resultados con AC y CC. Pinta estos cuantificadores en una gráfica como la vista anteriormente.

In [ ]:
from quantificationlib.bag_generator import PriorShift_BagGenerator
from quantificationlib.multiclass.em import EM
from quantificationlib.multiclass.df import DFy

n_bags = 50
bag_size = 500

# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------

### Ejercicio 2: Utiliza los 5000 ejemplos de entrenamiento para reentrenar tus cuatro cuantificadores. Usa estos cuantificadores para cuantificar las 1000 muestras presentes en el directorio T1.train_dev/T1/public/dev_samples. Visualizalo también gráficamente.
La prevalencia real de cada bag está en el archivo dev_prevalences.txt

In [ ]:
# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------

### Ejercicio 3 [Extra]. Hiperparámetros de los cuantificadores.
Algunos cuantificadores, como por ejemplo DFy tienen hiper-parámetros, como por ejemplo el número de bins o la función de distancia utilizada para comparar las distribuciones. Prueba a cambiar estos hiperparámetros y volver a ejecutar, ¿encuentras alguna diferencia? 

In [ ]:
# Tu código aquí
# -------------------------------------------------------------------------------------------------


# -------------------------------------------------------------------------------------------------

### Ejercicio 4 [Extra]. Usando un dataset multiclase.

Prueba a entrenar los cuantificadores utilizando un dataset multiclase: https://zenodo.org/records/11661820/files/T2.train_dev.zip?download=1. Todos los cuantificadores que hemos utilizado anteriormente son capaces también de resolver problemas multiclase.

